In [2]:
import torchaudio
import torch
from pathlib import Path


In [3]:

def split_wav(file_path, out_dir, chunk_duration=0.5, target_sr=16000):
    """
    Split a WAV file into fixed-length chunks and save them.

    Parameters:
        file_path (str or Path): Path to input WAV file.
        out_dir (str or Path): Directory to save chunks.
        chunk_duration (float): Duration of each chunk in seconds (default 0.5s).
        target_sr (int): Target sampling rate (default 16000 Hz).
    """
    file_path = Path(file_path)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Load audio
    waveform, sr = torchaudio.load(file_path)

    # Convert to mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample if needed
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(sr, target_sr)
        waveform = resampler(waveform)
        sr = target_sr

    num_samples_per_chunk = int(chunk_duration * sr)
    total_samples = waveform.shape[1]
    num_chunks = (total_samples + num_samples_per_chunk - 1) // num_samples_per_chunk

    # Split and save
    for i in range(num_chunks):
        start = i * num_samples_per_chunk
        end = min(start + num_samples_per_chunk, total_samples)
        chunk = waveform[:, start:end]

        # Pad last chunk if too short
        if chunk.shape[1] < num_samples_per_chunk:
            padding = num_samples_per_chunk - chunk.shape[1]
            chunk = torch.nn.functional.pad(chunk, (0, padding))

        out_file = out_dir / f"{file_path.stem}_chunk{i+1}.wav"
        torchaudio.save(str(out_file), chunk, sr)

    print(f"✅ Saved {num_chunks} chunks to {out_dir}")


In [8]:
# split_wav("/home/pierre/Downloads/drone-sound-128-ytshorts.savetube.me.wav", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/bb")
# split_wav("/home/pierre/Downloads/this-is-what-a-drone-sounds-like-128-ytshorts.savetube.me.wav", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")
# split_wav("/home/pierre/Downloads/this-drone-flight-is-unreal-128-ytshorts.savetube.me.wav", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")
# split_wav("/home/pierre/Downloads/drone-sound-effects-128-ytshorts.savetube.me.mp3", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")
split_wav("/home/pierre/Downloads/chilling-sound-of-a-uaf-drone-hunting-for-a-cowering-russian-soldier-128-ytshorts.savetube.me.mp3", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")

✅ Saved 30 chunks to /home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa


In [9]:
import os

def process_wav_folder(input_dir, output_dir):
    """
    Processes all .wav files in a given folder using split_wav(file_path, out_dir).

    Args:
        input_dir (str): Path to the folder containing .wav files.
        output_dir (str): Path to the folder where processed files will be saved.
    """

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Loop over all files in the input directory
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".wav"):
            file_path = os.path.join(input_dir, filename)
            print(f"Processing: {file_path}")

            try:
                split_wav(file_path, output_dir)
            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")

    print("✅ All .wav files processed successfully!")



In [11]:
process_wav_folder("/home/pierre/Downloads/download (2)/other", "/home/pierre/Downloads/download (2)/other_output")

Processing: /home/pierre/Downloads/download (2)/other/0.wav
✅ Saved 2 chunks to /home/pierre/Downloads/download (2)/other_output
Processing: /home/pierre/Downloads/download (2)/other/1.wav
✅ Saved 2 chunks to /home/pierre/Downloads/download (2)/other_output
Processing: /home/pierre/Downloads/download (2)/other/2.wav
✅ Saved 2 chunks to /home/pierre/Downloads/download (2)/other_output
Processing: /home/pierre/Downloads/download (2)/other/3.wav
✅ Saved 2 chunks to /home/pierre/Downloads/download (2)/other_output
Processing: /home/pierre/Downloads/download (2)/other/4.wav
✅ Saved 2 chunks to /home/pierre/Downloads/download (2)/other_output
Processing: /home/pierre/Downloads/download (2)/other/5.wav
✅ Saved 2 chunks to /home/pierre/Downloads/download (2)/other_output
Processing: /home/pierre/Downloads/download (2)/other/0 (2).wav
✅ Saved 2 chunks to /home/pierre/Downloads/download (2)/other_output
Processing: /home/pierre/Downloads/download (2)/other/1 (2).wav
✅ Saved 2 chunks to /home/pie

In [58]:
from datasets import load_dataset, ClassLabel, DatasetDict
ds = load_dataset("audiofolder", data_dir="../data/raw/test")
ds = DatasetDict({"test": ds["train"]})

In [57]:
from IPython.display import display, Audio
ad1 = ds["test"][200]
print(labels[ad1["label"]])
display(Audio(ad1["audio"]["array"], rate=ad1["audio"]["sampling_rate"]))

drone


In [59]:
ds.push_to_hub("Usernameeeeee/drone_test")

Map: 100%|██████████| 893/893 [00:00<00:00, 4025.97 examples/s]

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 37.25ba/s]
Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (1 / 1): 100%|██████████| 28.3MB / 28.3MB,  125kB/s  
Processing Files (1 / 1): 100%|██████████| 28.3MB / 28.3MB, 93.4kB/s  
New Data Upload: 100%|██████████| 74.7kB / 74.7kB, 93.4kB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.88s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Usernameeeeee/drone_test/commit/7cefdc478ba90423ef5e97ed1ad3de1d3aee2852', commit_message='Upload dataset', commit_description='', oid='7cefdc478ba90423ef5e97ed1ad3de1d3aee2852', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Usernameeeeee/drone_test', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Usernameeeeee/drone_test'), pr_revision=None, pr_num=None)